In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices("GPU")[0]
tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import cv2
import numpy as np

In [3]:
main = cv2.imread('data/green/189.jpg')

In [4]:
main.shape

(100, 46, 3)

In [3]:
data = tf.keras.preprocessing.image_dataset_from_directory("data/",shuffle = True, seed = 11,
                                                          image_size = (100, 50), 
                                                          batch_size = 32)

Found 450 files belonging to 3 classes.


In [4]:
class_names = data.class_names
class_names

['green', 'red', 'yellow']

In [5]:
def dataset_partitions(ds, train_split=0.8, val_split=0.1, test_split=0.1):
    
    ds_size = len(ds)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
train, val, test = dataset_partitions(data)

In [7]:
train = train.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
val = val.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
test = test.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(100, 50),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
])

In [9]:
NUM_CLASSES = len(data.class_names)
INPUT_SHAPE = (32, 100, 50, 3)

In [12]:
model = tf.keras.models.Sequential([
    resize_and_rescale,
    tf.keras.layers.Conv2D(16, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
])

In [13]:
model.build(INPUT_SHAPE)

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 100, 50, 3)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 98, 48, 16)          448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (32, 49, 24, 16)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (32, 47, 22, 32)          4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (32, 23, 11, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (32, 8096)                0         
_________________________________________________________________
dense_2 (Dense)              (32, 32)                 

In [21]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
history = model.fit(
    train,
    batch_size=32,
    validation_data=val,
    verbose=1,
    epochs=2,
)

Epoch 1/2
12/12 [==============================] - 6s 48ms/step - loss: 0.5741 - accuracy: 0.8121 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 2/2
12/12 [==============================] - 0s 8ms/step - loss: 0.0389 - accuracy: 0.9930 - val_loss: 1.0554e-04 - val_accuracy: 1.0000


In [18]:
scores = model.evaluate(test)
scores

2/2 [==============================] - 0s 45ms/step - loss: 5.8299e-04 - accuracy: 1.0000


[0.0005829940200783312, 1.0]

In [ ]:
m